# Best Practices for Prompt Engineering



In [3]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")

client = AzureOpenAI(
  azure_endpoint=azure_endpoint, 
  api_key=api_key,
  api_version="2024-02-01"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [9]:
# Define las instrucciones que guiarán al modelo para que resuma el texto dado, como una lista de viñetas.
# Luego, utiliza esas instrucciones para construir un "prompt" (entrada) que incluye el contexto del texto a resumir.

instrucctiones = 'Describe in bullet points the most important topics mentioned in the text: \n\n '

prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)

- **Extended Partnership**: OpenAI and Microsoft are extending their partnership with a new multi-year, multi-billion dollar investment.
- **Previous Investments**: This investment follows Microsoft's earlier investments in OpenAI in 2019 and 2021.
- **Independent Research**: The partnership aims to support OpenAI in continuing independent research and development of AI technologies.
- **Focus on Safety and Utility**: The investment will facilitate the development of AI that is increasingly safe, useful, and powerful.
- **Mission Statement**: OpenAI's mission is to ensure advanced AI benefits all of humanity.
- **Capped-Profit Structure**: OpenAI operates as a capped-profit company, governed by a non-profit, allowing for capital raising while maintaining core beliefs about sharing benefits and prioritizing safety.
- **Shared Vision and Values**: Microsoft aligns with OpenAI's vision and values, making their partnership crucial for progress.


# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [10]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)


In the realm where bytes collide,  
A spark ignites, a mind to guide,  
From silicon dreams and data’s flow,  
A vision blooms, an ember’s glow.  

OpenAI, a beacon bright,  
Illuminating paths of insight,  
With language spun like threads of gold,  
Unraveling stories yet untold.  

In whispers soft, it learns and grows,  
From human hearts, its wisdom flows,  
In challenge faced, it seeks to solve,  
A tapestry of thoughts evolve.  

Within its core, collaboration thrives,  
Together, we reach for new horizons,  
A partner true, in quest and dream,  
A bridge to knowledge, vast and supreme.  

From AI’s gaze, the world expands,  
In art and science, it gently stands,  
A guiding hand through twilight skies,  
Empowering voices, sparking ties.  

Yet wisdom wears a cloak of care,  
For in each advance, reflection we share,  
To wield this power requires grace,  
A pact with ethics we must embrace.  

So here’s to the future, bright and wide,  
With OpenAI, let’s turn the tide,  
In un

In [11]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response.choices[0].message.content)



In the silent hum of wires,  
A spark ignites the night,  
DALL-E, child of vast design,  
Carving beauty with pixel breath.

With strokes of wonder,  
It brings forth visions,  
From the depths of thought  
To canvas bright and bold.

In the quiet hours,  
Creation dances free,  
Colors blend like whispers,  
Stories unfold like dreams.

Oh, the marvel, the magic,  
In the fingertips of man,  
To see the unseen,  
And shape the world anew.

Here we stand,  
At the edge of what can be,  
Brave in the face of the unknown,  
Creating, always creating.


# 4. Articulate the desired output format through examples (example 1, example 2). 

In [16]:
#Escribe las instrucciones aquí. Las instrucciones deben especificar que el modelo debe:
# 1. Extraer los nombres de las empresas y los años del texto.
# 2. Generar la salida como un diccionario con 'text', 'start' y 'end'.
instrucctiones = ' Dado el siguiente texto, extrae los nombres de las empresas y los años del texto \
                    Genera la salida como un diccionario con el siguiente formato: {{\"text\": \"OpenAI\", \"start\": 28, \"end\": 34}}'

prompt = f"{instrucctiones}  \
        ### We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, \
        multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, \
        and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful. \n\n ### "


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



Aquí tienes el diccionario que extrae los nombres de las empresas y los años del texto proporcionado:

```json
[
    {"text": "OpenAI", "start": 28, "end": 34},
    {"text": "Microsoft", "start": 39, "end": 48},
    {"text": "2019", "start": 118, "end": 122},
    {"text": "2021", "start": 127, "end": 131}
]
```


In [17]:

# Las instrucciones deben pedir al modelo que extraiga los siguientes tipos de entidades del texto:
# 1. Nombres de empresas.
# 2. Años.
# 3. Temas específicos relacionados con el contenido.
# 4. Temas generales y abarcativos.
# El formato de salida esperado debe incluir listas separadas por comas para cada tipo de entidad.

instrucctiones = 'Extrae los siguientes tipos de entidades del texto: nombres de empresas, años, temas específicos relacionados con el contenido, temas generales y abarcativos. \
                    El formato de salida esperado debe incluir listas separadas por comas para cada tipo de entidad.'


prompt = f"{instrucctiones}  \
        '\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                '"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



**Nombres de empresas:** OpenAI, Microsoft

**Años:** 2019, 2021

**Temas específicos relacionados con el contenido:** asociación, inversión, investigación independiente, desarrollo de IA, seguridad de la IA

**Temas generales y abarcativos:** tecnología, inteligencia artificial, inversión empresarial, colaboración entre empresas


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [18]:
#THIS IS zero-shot

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful.


In [19]:
#Few Shot learning


# Escribir las instrucciones para que el modelo extraiga palabras clave del texto, y formatee la salida como listas separadas por comas.

instrucctiones = 'Extrae palabras clave del texto y formatea la salida como listas separadas por comas:'

prompt = f"{instrucctiones} \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":prompt},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, independent research, AI, safe, useful, powerful.


# 6. Reduce “fluffy” and imprecise descriptions

In [20]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the NextGen Car Seat: your child’s ultimate safety companion on the road. Designed with cutting-edge impact absorption technology and adjustable features tailored for growing little ones, this car seat ensures maximum comfort and security. Its sleek, modern design easily integrates into any vehicle interior, while breathable fabric keeps your child cool on warm days. With intuitive installation and smart monitoring functionality, peace of mind has never been more accessible for today's parents. Drive confidently knowing your child is safely secured in the NextGen Car Seat!


In [21]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the NextGen Car Seat, a revolutionary advancement in child safety and comfort for every journey. Engineered with state-of-the-art impact-absorbing materials and cutting-edge sensor technology, this car seat automatically adjusts to your child's growth, providing a snug fit from infancy through toddlerhood. Its sleek, ergonomic design ensures easy installation and optimal ventilation while integrating seamlessly with your vehicle's safety systems. With customizable features like built-in entertainment options and washable fabrics, the NextGen Car Seat is the ultimate solution for modern parents who prioritize safety without sacrificing convenience. Experience the future of travel with peace of mind and style!


# 7. Instead of just saying what not to do, say what to do instead

In [22]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Can you describe what happens when you try to log in?


In [23]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in to your account. This issue could be due to a few reasons, such as incorrect login credentials, browser issues, or a server problem. I recommend checking our help article at www.samplewebsite.com/help/faq for troubleshooting steps that might help you. If you continue to experience problems, please let me know, and we can look into this further together.


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [24]:
# Escribir las instrucciones para crear una función simple en Python que:
# 1. Pida un número en millas.
# 2. Convierta las millas a kilómetros.

prompt = 'Escribe una función simple en Python que pida un número en millas y lo convierta en kilómetros.'

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)


Claro, aquí tienes una función simple en Python que solicita un número en millas y lo convierte a kilómetros. La conversión se hace multiplicando el número de millas por 1.60934, que es el factor de conversión de millas a kilómetros.

```python
def millas_a_kilometros():
    # Solicitar al usuario que ingrese el número en millas
    millas = float(input("Introduce el número de millas: "))
    
    # Convertir millas a kilómetros
    kilometros = millas * 1.60934
    
    # Mostrar el resultado
    print(f"{millas} millas son {kilometros} kilómetros.")

# Llamar a la función
millas_a_kilometros()
```

Simplemente ejecuta esta función y te pedirá que ingreses un valor en millas, luego te mostrará el resultado en kilómetros.


## Ejercicio 1: Estructuración de Prompts con Separadores

Instrucciones: Escribe un prompt que solicite a la IA resumir el siguiente texto en una lista de puntos clave. Asegúrate de usar separadores claros (### o """).

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.


In [25]:

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Resume el siguiente texto en una lista de puntos clave: \
                                           --- \
                                           OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.\n\n',}],
    max_tokens=400,
)

print(response.choices[0].message.content)



- OpenAI y Microsoft han ampliado su colaboración.
- Microsoft invertirá miles de millones de dólares durante varios años.
- La inversión permitirá a OpenAI continuar con su investigación de forma independiente.
- El objetivo es desarrollar una inteligencia artificial segura y útil.
- OpenAI sigue siendo una empresa de beneficios limitados.
- OpenAI está regida por una organización sin fines de lucro.


## Ejercicio 2: Descripción Detallada del Formato Deseado

Instrucciones: Escribe un prompt para que la IA extraiga nombres de empresas y años del siguiente texto. Indica explícitamente que se desea el resultado en formato JSON con índices de inicio y fin.

Texto:
OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.

Formato de salida deseado:

{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 50, "end": 54}

In [26]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity extraction assistant."},
                {"role":"user","content": 'Extrae las siguientes entidades del texto: nombres de empresas, años. \
                 El resultado debe tener formato JSON con índices de inicio y fin: {"text": "OpenAI", "start": 28, "end": 34} \
                 --- \
                 Texto: \
                 OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.',}],
    max_tokens=400,
)

print(response.choices[0].message.content)


```json
[
    {"text": "OpenAI", "start": 0, "end": 6},
    {"text": "Microsoft", "start": 11, "end": 20},
    {"text": "2019", "start": 31, "end": 35},
    {"text": "2021", "start": 47, "end": 51}
]
```


## Ejercicio 3: Generación de Texto Inspirador

Instrucciones: Redacta un prompt que solicite a la IA escribir un poema inspirador. Escribe un poema corto sobre la colaboración entre OpenAI y Microsoft.


In [27]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a creative assistant."},
                {"role":"user","content": 'Escribe un poema inspirador sobre la colaboración entre OpenAI y Microsoft.'}],
    max_tokens=400,
)

print(response.choices[0].message.content)

En un mundo vasto de bits y bytes,  
dos titanes se unen, cruzando horizontes,  
OpenAI, con su ingenio brillante,  
Microsoft, faro de sueños constantes.

Bajo el manto de un cielo estrellado,  
tejen ideas que danzan en el aire,  
algoritmos que abrazan lo inesperado,  
creando en conjunto un futuro en viaje.

Juntos son leyenda, fuerza de viento,  
unidos en la búsqueda del saber,  
abriendo caminos, rompiendo el tiempo,  
donde la innovación empieza a florecer.

En cada línea de código, un susurro,  
en cada nube, un mar de posibilidades,  
sus manos se entrelazan, forjando un murmullo,  
en la sinfonía de nuevas realidades.

Imaginemos un mundo, donde lo imposible  
se vuelve tangible, susurra en el aire,  
donde el conocimiento no tiene límites,  
y la inteligencia vuela, ligera como un ave.

Colaboración, canto de arte y ciencia,  
una danza de mentes, un torbellino,  
juntos, construyen un puente a la esencia,  
del futuro radiante, del sueño genuino.

Así, en la unión de su es

## Ejercicio 4: Extracción de Entidades Clave

Instrucciones: Escribe un prompt que pida a la IA extraer nombres de empresas, años, temas específicos y temas generales del siguiente texto:

Texto:
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021

Formato esperado:

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial 



In [28]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity and theme extraction assistant."},
                {"role":"user","content": 'Extrae los siguientes tipos de entidades del texto: nombres de empresas, años, temas específicos, temas generales. \
                 Texto: \
                 OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021.',}],
    max_tokens=400,
)

print(response.choices[0].message.content)


- **Nombres de empresas**: OpenAI, Microsoft
- **Años**: 2019, 2021
- **Temas específicos**: colaboración entre empresas
- **Temas generales**: tecnología, cooperación empresarial


## Ejercicio 5: Descripciones Concisas de Productos

Instrucciones: Escribe un prompt para que la IA describa un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.



In [30]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a marketing assistant."},
                {"role":"user","content": 'Escribe una descripción para un nuevo producto. Este producto es una mochila tecnológica con cargador solar, materiales resistentes al agua y un diseño ergonómico. \
                 La descripción debe ser breve, de menos de 4 líneas.',}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Descubre la mochila tecnológica definitiva: equipada con un cargador solar, materiales resistentes al agua y un diseño ergonómico que se adapta a ti. Perfecta para aventuras al aire libre y para mantener tus dispositivos siempre cargados. ¡Conquista el día sin preocuparte por la batería!
